In [ ]:
import networkx as nx
import numpy as np
import pandas as pd

# Sample multiomics data (replace with your actual data)
gene_expression = pd.DataFrame({'Gene': ['A', 'B', 'C'], 'Expression': [10, 5, 12]})
protein_interaction = pd.DataFrame({'Protein1': ['A', 'B', 'A'], 'Protein2': ['B', 'C', 'C']})

# Create a graph
graph = nx.Graph()

# Add nodes with gene expression as features
for index, row in gene_expression.iterrows():
    graph.add_node(row['Gene'], expression=row['Expression'])

# Add edges from protein interaction data
for index, row in protein_interaction.iterrows():
    graph.add_edge(row['Protein1'], row['Protein2'])

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data

# Convert NetworkX graph to PyTorch Geometric Data
edge_index = torch.tensor(list(graph.edges)).t().contiguous()
node_features = torch.tensor([graph.nodes[node]['expression'] for node in graph.nodes]).unsqueeze(1).float() # Ensure correct shape
data = Data(x=node_features, edge_index=edge_index)

# Define a simple GCN model
class GCN(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

# Instantiate and train the model (example for node classification)
model = GCN(num_node_features=1, hidden_channels=16, num_classes=2) # Adjust num_classes based on task
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Create dummy labels for demonstration 
labels = torch.randint(0, 2, (data.num_nodes,))
data.y = labels

for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out, data.y)
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch}, Loss: {loss.item()}')